> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.2`: transfer learning mode demo (teachers forcing)
#### The *transfer learning* mode can be used for either
    1. Initial training of the Agent - where a newly built agent is trained from scratch while iterating through sufficiently large datasets over many epochs 
    2. Focusing of pre-trained Agent - where an already pre-trained agent is introduced to a small dataset for a small number of epochs.
In this notebook we are going to illustrate the first scenario. The provided dataset is processed by the workflow illustrated in the `Data Preparation` example. The required input is an empty model and the dataset with which this empty model is created.  
The Prior can be used afterwards for *reinforcement learning*, *tranfer learning* or just *sampling*.

## This is a rather slow process that depends on the number of epochs and dataset size

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.2")
output_dir = os.path.expanduser("~/Desktop/REINVENT_transfer_learning_initial_training_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## Setting up the configuration
`REINVENT` has an entry point that loads a specified `JSON` file on startup. `JSON` is a low-level data format that allows to specify a fairly large number of parameters in a cascading fashion very quickly. The parameters are structured into *blocks* which can in turn contain blocks or simple values, such as *True* or *False*, strings and numbers. In this tutorial, we will go through the different blocks step-by-step, explaining their purpose and potential values for given parameters. Note, that while we will write out the configuration as a `JSON` file in the end, in `python` we handle the same information as a simple `dict`.

In [2]:
# initialize the dictionary
configuration = {
    "version": 3,                          # we are going to use REINVENT's newest release
    "run_type": "transfer_learning",       # other run types: "scoring", "validation",
                                           #                  "transfer_learning",
                                           #                  "reinforcement_learning" and
                                           #                  "create_model"
    "model_type": "default"
}

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://127.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_path": os.path.join(output_dir, "progress.log"), # where the run's output is stored
    "job_name": "Transfer Learning demo", # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to "remote"
}

We will need to specify a path to an agent (parameter `model_path`), which can be a prior or trained agent. For the purpose of this notebook, we will use a prior shipped with the `REINVENT 3.2` repository.

The code block below will define the settings for `adaptive_lr_config` property of the configuration. These parameters are defining the behavior of the learning rate. Note that the mode is set to `"adaptive"`. We recommend adhering to the default values.

In [4]:
adaptive_lr_config = {
      "mode": "adaptive", # other modes: "exponential", "adaptive", "constant"
      "gamma": 0.8,
      "step": 1,
      "start": 5E-4,  # initial learning rate
      "min": 1E-5,
      "threshold": 1E-4,
      "average_steps": 4,
      "patience": 8,        # patience is the lower bound of how frequently the learning rate should change
      "restart_value": 1E-5,
      "sample_size": 10000,  # this is relevant for stats and decision on how to update the learning rate
      "restart_times": 0
    }

In [5]:
input_SMILES_path = os.path.join(ipynb_path, "data/chembl.filtered.smi") 
input_model_path = os.path.join(ipynb_path, "models/random.prior.new")
output_model_path = os.path.join(output_dir, "chembl.prior")
# The final focused agent will be named "chembl.prior"
# The intermediate steps will be named "chembl.prior.1", "chembl.prior.2", "chembl.prior.3" and etc.

# add the "parameters" block
configuration["parameters"] = {
    "input_model_path": input_model_path,          # path to prior or empty model
    "output_model_path": output_model_path,        # location to store the chembl prior
    "input_smiles_path": input_SMILES_path,        # path to input smiles
    
    "save_every_n_epochs": 1,      # how often to save the Prior. Here its stored after each epoch
    "batch_size": 128,             # batch size the input data
    "num_epochs": 3,               # number of epochs to train Prior (set to 3 in this notebook for illustration).
                                   # in reality, 20 epochs could result in a sufficiently good Prior
                                   # NOTE: 20 epochs could take days to compute
    "standardize": False,          # we assume all SMILES strings have been pre-processed
    "randomize": True,             # this triggers data augmentation and will slow down the training a bit.
    "adaptive_lr_config": adaptive_lr_config        # setting the learning rate behavior
}

In [6]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "transfer_learning_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## Run `REINVENT`
Now it is time to execute `REINVENT` locally. 
This training might take days with the suggested dataset and the number of epochs.
Best execute for just a couple of epoch to optain a realistic estimate.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent.v3.2

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [7]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [8]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

## Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_transfer_learning_demo

# make sure, you have activated the proper environment
conda activate reinvent.v3.2

# start tensorboard
tensorboard --logdir progress.log
```
